# Subjects Evaluation

Here we obtain the performance of the best solutions found by GenFeatS-BS and compare them to the performance obtained from a standard feature extraction procedure.

**GenFeatS-BS**

This method applies a Genetic Algorithm design that finds the best combination of channel(s), frequency band(s), feature and feature parameters for a given subject.


**Comparison Method**

This method creates a reduced filter bank of the subject's signals, using mu and beta frequency bands, then ICA method is applied to the the signals and from the result, CSP method is used for feature extraction.

In [1]:
import mne
import numpy as np
from sklearn import svm
from mne.decoding import CSP
from mne.preprocessing import ICA
from src.genfeats.genfeats_bs import GenFeatSBS
from src.genfeats.dna.chromesome import Chromesome
from sklearn.model_selection import cross_val_score
from src.sighandling.sighandling import get_dataset_bbcic4_2b

In [2]:
# GenFeatS-BS resources
resources_folder = 'D:/Dev/GenFeatS-BS-Study/resources/'
results_path = 'D:/Dev/GenFeatS-BS-Study/data/processed/'
execution_metadata = {
    'subject': 'S',
    'survival_rate': 0.1,
    'chromesome_size': 4,
    'population_size': 40,
    'extintions_limit': 10,
    'generations_limit': 400
}

In [3]:
# Define the SVM classifier
classifier = svm.SVC(kernel='rbf')

In [41]:
def cross_validate(classifier, X, y):
    # Perform cross-validation
    scores = cross_val_score(classifier, X, y)

    # Print the mean accuracy across folds
    print("Accuracy: %0.4f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [5]:
def get_filtered_epochs(epochs):
    channels = epochs.ch_names
    mu_channels = {ch:f'{ch}_mu' for ch in channels}
    beta_channels = {ch:f'{ch}_beta' for ch in channels}
    
    # Apply the filter to the data
    epochs_mu_filt = epochs.copy().filter(l_freq=8, h_freq=13, picks='eeg', method = 'iir')
    epochs_mu_filt.rename_channels(mu_channels)
    
    # Apply the filter to the data
    epochs_beta_filt = epochs.copy().filter(l_freq=13, h_freq=30, picks='eeg', method = 'iir')
    epochs_beta_filt.rename_channels(beta_channels)
    
    return epochs_mu_filt, epochs_beta_filt
    

In [6]:
def apply_ICA(epochs_mu_filt, epochs_beta_filt):
    # Create an ICA object
    ica = ICA(n_components=0.9)

    ### MU
    # Fit the ICA object to the data and extract the ICs
    ica.fit(epochs_mu_filt)
    # Apply the ICA to the data to obtain the IC activations
    ica_epochs_mu = ica.apply(epochs_mu_filt)

    ## BETA
    # Fit the ICA object to the data and extract the ICs
    ica.fit(epochs_beta_filt)
    # Apply the ICA to the data to obtain the IC activations
    ica_epochs_beta = ica.apply(epochs_beta_filt)  
    
    return ica_epochs_mu, ica_epochs_beta

In [7]:
def extract_CSP_features(epochs, ica_epochs_mu, ica_epochs_beta, y):
    epochs_final = epochs.copy()
    epochs_final.add_channels([ica_epochs_mu, ica_epochs_beta], force_update_info=True)
    epochs_final.drop_channels(epochs.ch_names)
    
    # Create a CSP object
    csp = CSP()

    # Apply CSP to the epochs data
    X_csp = csp.fit_transform(epochs_final.get_data(), y)
    
    return X_csp

In [8]:
def get_standard_features(epochs, y):
    
    epochs_mu_filt, epochs_beta_filt = get_filtered_epochs(epochs)
    
    ica_epochs_mu, ica_epochs_beta = apply_ICA(epochs_mu_filt, epochs_beta_filt)
    
    X_csp = extract_CSP_features(epochs, ica_epochs_mu, ica_epochs_beta, y)
    
    return X_csp


## S1

In [9]:
# Load the epochs data
data_file = 'D:/Dev/GenFeatS-BS-Study/data/external/segmented/S1_clean.mat'
epochs = get_dataset_bbcic4_2b(data_file)

Not setting metadata
159 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
165 matching events found
No baseline correction applied
0 projection items activated
Dropped 0 epochs: 
Dropped 6 epochs: 159, 160, 161, 162, 163, 164
Not setting metadata
318 matching events found
No baseline correction applied


In [10]:
# GenFeatS-BS Instance

genfeatsbs = GenFeatSBS(
    resources_folder, 
    epochs, 
    classifier, 
    results_path=results_path, 
    execution_metadata=execution_metadata,
    chromesome_size = 4,
    generations_limit = 100
)

2023-04-08 21:07:24,062	INFO worker.py:1518 -- Started a local Ray instance.


**S1 Chromesome**

In [11]:
s1_solution_dict = {
    'g1': { 
        'feature': 'spect_edge_freq',
        'frequency_bands': [[0.5, 4], [7.5, 13]],
        'channels': ['C4'],
        'feature_parameters': {
            "psd_method": "multitaper",
            "ref_freq": 83,
            "edge": 0.6
        }
    },
    'g2': { 
        'feature': 'hjorth_mobility_spect',
        'frequency_bands': [[17.5,24]],
        'channels': ['C3', 'Cz'],
        'feature_parameters': {
            "psd_method": "fft"
        }
    },
    'g3': { 
        'feature': 'line_length',
        'frequency_bands': [[7.5,13]],
        'channels': ['C3', 'C4'],
        'feature_parameters': {}
    },
}

In [12]:
solution = Chromesome.from_dict(s1_solution_dict)
X_s1 = genfeatsbs.map.to_phenotype(solution)
cross_validate(classifier, X_s1, epochs.events[:, -1])

Accuracy: 0.74 (+/- 0.15)


#### Standard Method

In [13]:
X_csp = get_standard_features(epochs, epochs.events[:, -1])
cross_validate(classifier, X_csp, epochs.events[:, -1])

Setting up band-pass filter from 8 - 13 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 8.00, 13.00 Hz: -6.02, -6.02 dB

Setting up band-pass filter from 13 - 30 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 13.00, 30.00 Hz: -6.02, -6.02 dB

Fitting ICA to data using 3 channels (please be patient, this may take a while)
Selecting by explained variance: 3 components
Fitting ICA took 0.1s.
Applying ICA to Epochs instance
    Transforming to ICA space (3 components)
    Zeroing out 0 ICA components
    Projecting back using 3 PCA components
Fitting ICA to data using 3 channels (please be patient, this may take a while)
Selecting by explained variance: 3 components
Fitting ICA took 0.3s.
Applyin

## S2

In [14]:
# Load the epochs data
data_file = 'D:/Dev/GenFeatS-BS-Study/data/external/segmented/S2_clean.mat'
epochs = get_dataset_bbcic4_2b(data_file)

Not setting metadata
163 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
166 matching events found
No baseline correction applied
0 projection items activated
Dropped 0 epochs: 
Dropped 3 epochs: 163, 164, 165
Not setting metadata
326 matching events found
No baseline correction applied


In [15]:
# GenFeatS-BS Instance

genfeatsbs = GenFeatSBS(
    resources_folder, 
    epochs, 
    classifier, 
    results_path=results_path, 
    execution_metadata=execution_metadata,
    chromesome_size = 4,
    generations_limit = 100
)

**S2 Chromesome**

In [16]:
s2_solution_dict = {
    'g1': { 
        'feature': 'hjorth_complexity_spect',
        'frequency_bands': [[7.5,13]],
        'channels': ['C3', 'C4'],
        'feature_parameters': {
            "psd_method": "multitaper"
        }
    },
    'g2': { 
        'feature': 'spect_edge_freq',
        'frequency_bands': [[7.5, 22], [13, 30]],
        'channels': ['C3'],
        'feature_parameters': {
            "psd_method": "fft",
            "ref_freq": 125.0,
            "edge": 0.7
        }
    },
    'g3': { 
        'feature': 'samp_entropy',
        'frequency_bands': [[7.5,13]],
        'channels': ['C3', 'C4'],
        'feature_parameters': {
            "emb": 16,
            "metric": "infinity"
        }
    }
}

In [17]:
solution = Chromesome.from_dict(s2_solution_dict)
X_s2 = genfeatsbs.map.to_phenotype(solution)
cross_validate(classifier, X_s2, epochs.events[:, -1])

Accuracy: 0.67 (+/- 0.02)


#### Standard Method

In [18]:
X_csp = get_standard_features(epochs, epochs.events[:, -1])
cross_validate(classifier, X_csp, epochs.events[:, -1])

Setting up band-pass filter from 8 - 13 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 8.00, 13.00 Hz: -6.02, -6.02 dB

Setting up band-pass filter from 13 - 30 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 13.00, 30.00 Hz: -6.02, -6.02 dB

Fitting ICA to data using 3 channels (please be patient, this may take a while)
Selecting by explained variance: 2 components
Fitting ICA took 0.2s.
Applying ICA to Epochs instance
    Transforming to ICA space (2 components)
    Zeroing out 0 ICA components
    Projecting back using 3 PCA components
Fitting ICA to data using 3 channels (please be patient, this may take a while)
Selecting by explained variance: 2 components
Fitting ICA took 0.1s.
Applyin

## S3

In [19]:
# Load the epochs data
data_file = 'D:/Dev/GenFeatS-BS-Study/data/external/segmented/S3_clean.mat'
epochs = get_dataset_bbcic4_2b(data_file)

Not setting metadata
146 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
150 matching events found
No baseline correction applied
0 projection items activated
Dropped 0 epochs: 
Dropped 4 epochs: 146, 147, 148, 149
Not setting metadata
292 matching events found
No baseline correction applied


In [20]:
# GenFeatS-BS Instance

genfeatsbs = GenFeatSBS(
    resources_folder, 
    epochs, 
    classifier, 
    results_path=results_path, 
    execution_metadata=execution_metadata,
    chromesome_size = 4,
    generations_limit = 100
)

**S3 Chromesome**

In [21]:
s3_solution_dict = {
    'g1': { 
        'feature': 'samp_entropy',
        'frequency_bands': [[7.5,13]],
        'channels': ['C3', 'C4'],
        'feature_parameters': {
            "emb": 6,
            "metric": "infinity"
        }
    },
    'g2': { 
        'feature': 'kurtosis',
        'frequency_bands': [[0.5, 4], [4, 7.5]],
        'channels': ['C4'],
        'feature_parameters': {}
    },
    'g3': { 
        'feature': 'hjorth_complexity_spect',
        'frequency_bands': [[7.5,13]],
        'channels': ['C3','C4'],
        'feature_parameters': {
            "psd_method": "fft"
        }
    }
}

In [22]:
solution = Chromesome.from_dict(s3_solution_dict)
X_s3 = genfeatsbs.map.to_phenotype(solution)
cross_validate(classifier, X_s3, epochs.events[:, -1])

Accuracy: 0.65 (+/- 0.08)


#### Standard Method

In [23]:
X_csp = get_standard_features(epochs, epochs.events[:, -1])
cross_validate(classifier, X_csp, epochs.events[:, -1])

Setting up band-pass filter from 8 - 13 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 8.00, 13.00 Hz: -6.02, -6.02 dB

Setting up band-pass filter from 13 - 30 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 13.00, 30.00 Hz: -6.02, -6.02 dB

Fitting ICA to data using 3 channels (please be patient, this may take a while)
Selecting by explained variance: 2 components
Fitting ICA took 0.1s.
Applying ICA to Epochs instance
    Transforming to ICA space (2 components)
    Zeroing out 0 ICA components
    Projecting back using 3 PCA components
Fitting ICA to data using 3 channels (please be patient, this may take a while)
Selecting by explained variance: 2 components
Fitting ICA took 0.2s.
Applyin

## S4

In [24]:
# Load the epochs data
data_file = 'D:/Dev/GenFeatS-BS-Study/data/external/segmented/S4_clean.mat'
epochs = get_dataset_bbcic4_2b(data_file)

Not setting metadata
201 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
198 matching events found
No baseline correction applied
0 projection items activated
Dropped 3 epochs: 198, 199, 200
Dropped 0 epochs: 
Not setting metadata
396 matching events found
No baseline correction applied


In [25]:
# GenFeatS-BS Instance

genfeatsbs = GenFeatSBS(
    resources_folder, 
    epochs, 
    classifier, 
    results_path=results_path, 
    execution_metadata=execution_metadata,
    chromesome_size = 4,
    generations_limit = 100
)

**S4 Chromesome**

In [39]:
s4_solution_dict = {
    'g1': { 
        'feature': 'hjorth_mobility_spect',
        'frequency_bands': [[7.5,22]],
        'channels': ['C4', 'Cz'],
        'feature_parameters': {
            'psd_method': 'welch'
        }
    },
    'g2': { 
        'feature': 'hjorth_complexity_spect',
        'frequency_bands': [[7.5,13]],
        'channels': ['C3', 'C4'],
        'feature_parameters': {
            'psd_method': 'fft'
        }
    },
    'g3': { 
        'feature': 'hjorth_complexity',
        'frequency_bands': [[13,17.5]],
        'channels': ['C4'],
        'feature_parameters': {}
    }
}

In [42]:
solution = Chromesome.from_dict(s4_solution_dict)
X_s4 = genfeatsbs.map.to_phenotype(solution)
cross_validate(classifier, X_s4, epochs.events[:, -1])

Accuracy: 0.8662 (+/- 0.11)


#### Standard Method

In [ ]:
X_csp = get_standard_features(epochs, epochs.events[:, -1])
cross_validate(classifier, X_csp, epochs.events[:, -1])